In [1]:
!pip install -q datasets
!pip install -q bitsandbytes
!pip install -q peft
!pip install -q accelerate
!pip install -q trl
!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 2

In [ ]:
# !pip install --upgrade pyarrow

In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from ast import literal_eval

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# General parameters
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # The model that you want to train from the Hugging Face hub
new_model = "llamma-seizure-json-generator"  # The name for fine-tuned LoRA Adaptor

In [5]:
# LoRA parameters
lora_r = 32
lora_alpha = lora_r * 2
lora_dropout = 0.1
target_modules = ["q_proj", "v_proj", "k_proj"]

In [6]:
# QLoRA parameters
load_in_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
bnb_4bit_use_double_quant = False

In [7]:
# TrainingArguments parameters
num_train_epochs = 10
fp16 = False
bf16 = False
per_device_train_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
learning_rate = 0.0015
weight_decay = 0.01
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 10

# SFT parameters
max_seq_length = None
packing = False
device_map = {"": 0}

# Dataset parameters
use_special_template = True
response_template = " ### Answer:"
instruction_prompt_template = "### Human:"
use_llama_like_model = True

In [8]:
def load_data(file_path):
    percent_of_train_dataset = 0.90
    data = load_dataset("json", data_files=file_path, split="train")

    split_dataset = data.train_test_split(
        train_size=int(data.num_rows * percent_of_train_dataset), seed=19, shuffle=False
    )
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]
    print(f"Size of the train set: {len(train_dataset)}. Size of the validation set: {len(eval_dataset)}")
    return train_dataset, eval_dataset

dataset_dict = {"seizure": {}, "prescription": {}}
# dataset_dict["prescription"]["train_dataset"], dataset_dict["prescription"]["eval_dataset"] = load_data(
#     file_path="/content/prescription.jsonl")

dataset_dict["seizure"]["train_dataset"], dataset_dict["seizure"]["eval_dataset"] = load_data(
    file_path="/content/seizure_frequency.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

Size of the train set: 236. Size of the validation set: 27


In [9]:
# Load LoRA configuration
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

In [10]:
# Load QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
)

In [16]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config, device_map=device_map)
model.config.use_cache = False

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [11]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=new_model,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    gradient_checkpointing=gradient_checkpointing,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    do_eval=True,
    evaluation_strategy="steps"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training
if not tokenizer.chat_template:
    tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [13]:
def special_formatting_prompts(example):
    output_texts = []
    for i in range(len(example["instruction"])):
        text = f"{instruction_prompt_template}{example['instruction'][i]}\n{response_template} {example['output'][i]}"
        output_texts.append(text)
    return output_texts


def normal_formatting_prompts(example):
    output_texts = []
    for i in range(len(example["instruction"])):
        chat_temp = [
            {"role": "system", "content": example["instruction"][i]},
            {"role": "assistant", "content": example["output"][i]},
        ]
        text = tokenizer.apply_chat_template(chat_temp, tokenize=False)
        output_texts.append(text)
    return output_texts

In [14]:
if use_special_template:
    formatting_func = special_formatting_prompts
    if use_llama_like_model:
        response_template_ids = tokenizer.encode(response_template, add_special_tokens=False)[2:]
        collator = DataCollatorForCompletionOnlyLM(response_template=response_template_ids, tokenizer=tokenizer)
    else:
        collator = DataCollatorForCompletionOnlyLM(response_template=response_template, tokenizer=tokenizer)
else:
    formatting_func = normal_formatting_prompts

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_dict["seizure"]["train_dataset"],
    eval_dataset=dataset_dict["seizure"]["eval_dataset"],
    peft_config=peft_config,
    formatting_func=formatting_func,
    data_collator=collator,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/236 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

In [18]:
if torch.cuda.device_count() > 1:
  model.is_parallelizable = True
  model.model_parallel = True

# Train model
trainer.train()

# Save fine tuned Lora Adaptor
trainer.model.save_pretrained(new_model)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
10,1.133500,0.383794
20,0.270300,0.256186
30,0.200300,0.144500
40,0.167000,0.205004
50,0.178600,0.190613
60,0.452400,0.200158
70,0.177600,0.183814
80,0.186700,0.151448
90,0.214900,0.318942
100,0.224200,0.161213


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

In [ ]:
# import torch
# import gc


# def clear_hardwares():
#     torch.clear_autocast_cache()
#     torch.cuda.ipc_collect()
#     torch.cuda.empty_cache()
#     gc.collect()


# clear_hardwares()
# clear_hardwares()

In [19]:
def generate(model, prompt: str, kwargs):
    tokenized_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)

    prompt_length = len(tokenized_prompt.get("input_ids")[0])

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**tokenized_prompt, **kwargs) if kwargs else model.generate(**tokenized_prompt)
        output = tokenizer.decode(output_tokens[0][prompt_length:], skip_special_tokens=True)

    return output

In [20]:
new_model

'llamma-seizure-json-generator'

In [21]:
new_model_path = "/content/llamma-seizure-json-generator"

In [22]:
ft_model = AutoModelForCausalLM.from_pretrained(new_model_path)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [23]:
peft_model = PeftModel.from_pretrained(ft_model, new_model)
# del base_model

In [24]:
sample = dataset_dict["seizure"]["eval_dataset"][8]
if use_special_template:
    prompt = f"{instruction_prompt_template}{sample['instruction']}\n{response_template}"
else:
    chat_temp = [{"role": "system", "content": sample["instruction"]}]
    prompt = tokenizer.apply_chat_template(chat_temp, tokenize=False, add_generation_prompt=True)

prompt

"### Human:Your task is to extract prescription information from a clinical text\n\nBelow is the clinical notes from a doctor, delimited by triple quotes.\nclinical text: ```Diagnosis: temporal lobe epilepsy right hippocampal sclerosis right hippocampectomy 2018  Current anti-seizure medication: lamotrigine 200mg bd  Seizure type and frequency:   focal seizures with altered awareness   (deja-vu), last event 2018   focal to bilateral convulsive seizures  , last event 2018  Previous investigations include: MRI 2018 right hippocampal sclerosis Video EEG 2018, 2 seizures captured with right temporal lobe focus  Dear doctor,  I reviewed this 48 year old gentleman for thi first time in clinic today.  As you know he had 3 febrile seizures between the ages of 3 and 5 and then started developing focal onset seizures from the age of 6. They continued until his surgery a few years ago. I was very pleased to note that he remains   seizure free   after his surgery. Before the surgery he was having 

In [25]:
gen_kwargs = {"max_new_tokens": 128}
generated_texts = generate(model=peft_model, prompt=prompt, kwargs=gen_kwargs)
print(generated_texts)

<ipython-input-19-cac47dd2b0e1>:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 {'start_index': '512', 'end_index': '520', 'text':'seizure free', 'attributes': {'LowerNumberOfSeizures': None, 'UpperNumberOfSeizures': None, 'TimePeriod': None, 'NumberOfTimePeriods': None, 'MonthDate': None, 'TimeSince_or_TimeOfEvent': 'Since', 'NumberOfSeizures': '0', 'PointInTime': 'LastClinic', 'FrequencyChange': None, 'YearDate': None, 'LowerNumberOfTimePeriods': None, 'UpperNumberOfTimePeriods': None, 'AgeUnit':


In [26]:
sample["output"]

{'start_index': '240',
 'end_index': '278',
 'text': 'focal to bilateral convulsive seizures',
 'attributes': {'LowerNumberOfSeizures': None,
  'UpperNumberOfSeizures': None,
  'TimePeriod': None,
  'NumberOfTimePeriods': None,
  'MonthDate': None,
  'TimeSince_or_TimeOfEvent': 'Since',
  'NumberOfSeizures': '0',
  'PointInTime': None,
  'FrequencyChange': None,
  'YearDate': '2018',
  'LowerNumberOfTimePeriods': None,
  'UpperNumberOfTimePeriods': None,
  'AgeUnit': None,
  'AgeLower': None,
  'AgeUpper': None,
  'DayDate': None,
  'DiagCategory': None,
  'Certainty': None,
  'Negation': None}}

In [27]:
from tqdm.notebook import tqdm

In [28]:
progress_bar = tqdm(total=len(dataset_dict["seizure"]["eval_dataset"]))
generated_texts_list = []
for eval_sample in dataset_dict["seizure"]["eval_dataset"]:
  chat_temp = [{"role": "system", "content": eval_sample["instruction"]}]
  prompt = tokenizer.apply_chat_template(chat_temp, tokenize=False,
                                           add_generation_prompt=True)
  generated_texts_list.append(generate(model=peft_model, prompt=prompt,
                                       kwargs=gen_kwargs))
  progress_bar.update(1)
progress_bar.close()

  0%|          | 0/27 [00:00<?, ?it/s]

<ipython-input-19-cac47dd2b0e1>:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setti

In [29]:
import pandas as pd

In [30]:
tmp = dataset_dict["seizure"]["eval_dataset"].to_dict()
tmp = pd.DataFrame.from_dict(tmp)

tmp.head()

,instruction,output
0,Your task is to extract prescription informati...,"{'start_index': '1060', 'end_index': '1096', '..."
1,Your task is to extract prescription informati...,"{'start_index': '352', 'end_index': '358', 'te..."
2,Your task is to extract prescription informati...,"{'start_index': '851', 'end_index': '871', 'te..."
3,Your task is to extract prescription informati...,"{'start_index': '270', 'end_index': '279', 'te..."
4,Your task is to extract prescription informati...,"{'start_index': '232', 'end_index': '239', 'te..."


In [31]:
tmp["generated_text"] = generated_texts_list

tmp.head()

,instruction,output,generated_text
0,Your task is to extract prescription informati...,"{'start_index': '1060', 'end_index': '1096', '...","{'start_index': '512', 'end_index': '520', 'te..."
1,Your task is to extract prescription informati...,"{'start_index': '352', 'end_index': '358', 'te...","{'start_index': '566', 'end_index': '573', 'te..."
2,Your task is to extract prescription informati...,"{'start_index': '851', 'end_index': '871', 'te...","{'start_index': '566', 'end_index': '573', 'te..."
3,Your task is to extract prescription informati...,"{'start_index': '270', 'end_index': '279', 'te...","{'start_index': '349', 'end_index': '357', 'te..."
4,Your task is to extract prescription informati...,"{'start_index': '232', 'end_index': '239', 'te...","{'start_index': '362', 'end_index': '390', 'te..."


In [32]:
tmp.to_csv("seizure_output.csv")

In [33]:
from google.colab import files
files.download('/content/seizure_output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>